# Semantic similarity

Hiermit messe ich den Winkel zweier Vektoren. Je geringer der Winkel, desto ähnlicher sind sich die Begriffe hinter den Vektoren.

Die Testdaten wurden von mir und einem LLM generiert.


## Evaluation

In [1]:
import tqdm
import pandas
from sentence_transformers import SentenceTransformer, util

models = {
    'GerMedBERT': 'GerMedBERT/medbert-512-no-duplicates',
    'E5-Large': 'intfloat/multilingual-e5-large',
    'GBERT-Large': 'deepset/gbert-large',
    'German-MedBERT': 'smanjil/German-MedBERT',
}

testdata = [
    ('Medikamentenkühlschrank', 'Pharmazeutische Lagerung'),
    ('Temperaturüberwachung', 'Temperaturmonitoring'),
    ('Impfstoffe', 'Vakzine'),
    ('DIN 13277', 'Medizingerät-Norm'),
    ('Temperaturkonstanz', 'Temperaturstabilität'),
    ('Kühlsystem', 'Kühltechnik'),
    ('Gradgenaue Temperaturregelung', 'Präzise Temperaturkontrolle'),
    ('Temperaturprofil', 'Temperaturverlauf'),
    ('Optische und akustische Alarme', 'Visuelles und auditives Warnsystem'),
    ('Alarmhistorie', 'Alarm-Protokoll'),
    ('Netzausfallalarm', 'Stromausfall-Warnung'),
    ('SafetyDevice', 'Sicherheitsvorrichtung'),
    ('Außenmaße', 'Abmessungen', 'Dimensionen'),
    ('Brutto-/Nettorauminhalt', 'Fassungsvermögen', 'Volumen'),
    ('Energieverbrauch', 'Stromverbrauch'),
    ('Belastbarkeit', 'Traglast'),
    ('USB-Schnittstelle', 'USB-Anschluss'),
    ('WiFi/LAN-Schnittstelle', 'Netzwerk-Verbindung'),
    ('SmartMonitoring', 'Intelligente Überwachung'),

    ('Kühlmittel', 'Kältemittel'),
    ('Kühlaggregate', 'Kühlmaschinen'),
    ('Kühlkettenüberwachung', 'Temperaturüberwachung'),
    ('Lagerbedingungen', 'Lageranforderungen'),
    ('Medizinprodukte', 'Gesundheitsprodukte'),
    ('Pharmazeutische Produkte', 'Medikamente'),
]

all_results = []


/home/bruce/.local/share/mise/installs/python/3.13.7/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**Evaluation**

Als Cousin Sililarity

In [2]:
def evaluate_semantic_similarity(model, testdata, name):
    results = []
    for pair in tqdm.tqdm(testdata, desc="Testing pairs: "):
        emb1 = model.encode(pair[0])
        emb2 = model.encode(pair[1])
        similarity = util.cos_sim(emb1, emb2).item()
        results.append({
            'term1': pair[0],
            'term2': pair[1],
            'model': name,
            'similarity': similarity
        })
    
    return pandas.DataFrame(results)

In [3]:
for model_name, model_path in tqdm.tqdm(models.items(), desc="Testing models: "):
    model = SentenceTransformer(model_path)
    model_results = evaluate_semantic_similarity(model, testdata, model_name)

    all_results.append(model_results)

Testing models:   0%|          | 0/4 [00:00<?, ?it/s]No sentence-transformers model found with name GerMedBERT/medbert-512-no-duplicates. Creating a new one with mean pooling.
Some weights of BertModel were not initialized from the model checkpoint at GerMedBERT/medbert-512-no-duplicates and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Testing models: 100%|██████████| 4/4 [01:06<00:00, 16.67s/it]


## Similarity-Score

In [4]:
final = pandas.concat(all_results, ignore_index=True)
# final.sort_values('similarity', ascending=False)

comparsion = final.pivot_table(
    index=['term1', 'term2'],
    columns='model',
    values='similarity',
    margins=True,
    margins_name="Mean"
)

comparsion

,model,E5-Large,GBERT-Large,GerMedBERT,German-MedBERT,Mean
term1,term2,,,,,
Alarmhistorie,Alarm-Protokoll,0.897827,0.923426,0.642584,0.825630,0.822367
Außenmaße,Abmessungen,0.920419,0.891052,0.572276,0.646247,0.757499
Belastbarkeit,Traglast,0.847038,0.914721,0.624196,0.733079,0.779758
Brutto-/Nettorauminhalt,Fassungsvermögen,0.844331,0.871637,0.499751,0.778013,0.748433
DIN 13277,Medizingerät-Norm,0.822445,0.865036,0.613983,0.761719,0.765796
Energieverbrauch,Stromverbrauch,0.979007,0.972403,0.865009,0.957506,0.943481
Gradgenaue Temperaturregelung,Präzise Temperaturkontrolle,0.967811,0.931056,0.682368,0.898669,0.869976
Impfstoffe,Vakzine,0.894577,0.874931,0.840286,0.638070,0.811966
Kühlaggregate,Kühlmaschinen,0.889404,0.943981,0.812491,0.681548,0.831856
